# Stochastic Model - Geometric Brownian Motion (GBM)

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import shapiro, jarque_bera


In [ ]:
# Define the assets
assets = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'TSLA', 'SPY', 'AGG', 'PSP', 'VNQ', 'SLV', 'GLD', 'USO', 'DJI']
cryptos = ['BTC-USD', 'ETH-USD', 'SOL-USD', 'BNB-USD', 'XRP-USD', 'TON-USD', 'DOGE-USD', 'ADA-USD', 'SHIB-USD', 'AVAX-USD']

# Define the start and end dates
start_date = '2022-01-01'
end_date = '2024-01-01'

### Functions

In [ ]:
# Function to retrieve historical price data
def get_historical_prices(tickers, start, end):
    data = yf.download(tickers, start=start, end=end)['Adj Close']
    return data

# Function to implement GBM
def geometric_brownian_motion(S0, mu, sigma, dt, N):
    dt_sigma = sigma * np.sqrt(dt)
    return S0 * np.exp(np.cumsum((mu - 0.5 * sigma**2) * dt + dt_sigma * np.random.randn(T, N), axis=0))

# Function to perform statistical tests
def perform_statistical_tests(returns):
    shapiro_stat, shapiro_p = shapiro(returns)
    jarque_bera_stat, jarque_bera_p = jarque_bera(returns)
    return shapiro_p, jarque_bera_p

In [ ]:
# Retrieve historical prices and perform GBM simulation for each asset
for asset in assets + cryptos:
    # Retrieve historical prices
    data = get_historical_prices(asset, start_date, end_date)
    
    # Check if data is empty
    if data.empty:
        print(f"No data available for {asset} within the specified date range.")
        continue
    
    # Fill missing values and drop NaNs
    data = data.ffill().dropna()
    
    # Convert Series to DataFrame if necessary
    if isinstance(data, pd.Series):
        data = pd.DataFrame(data, columns=[asset])
    
    # Check if daily returns are empty
    daily_returns = data.pct_change().dropna()
    if daily_returns.empty:
        print(f"No daily returns available for {asset}.")
        continue
    
    # Parameter estimation
    mu = daily_returns.mean()
    sigma = daily_returns.std()
    
    # GBM simulation
    T = len(data)
    N = 100
    dt = 1 / 252  # trading days per year
    S0 = np.tile(data.iloc[-1].values, (N, 1)).T  # Repeat S0 N times
    mu_tile = np.tile(mu.values.reshape(-1, 1), (1, N))
    sigma_tile = np.tile(sigma.values.reshape(-1, 1), (1, N))
    gbm = geometric_brownian_motion(S0, mu_tile, sigma_tile, dt, N)
    
    # Statistical tests
    shapiro_p, jarque_bera_p = perform_statistical_tests(daily_returns)
    
    # Print data and statistical test results
    print(f"Asset: {asset}")
    print("Historical Prices:")
    print(data)
    print("\nGBM Simulation:")
    print(gbm)
    print("\nStatistical Test Results:")
    print(f"Shapiro-Wilk p-value: {shapiro_p}")
    print(f"Jarque-Bera p-value: {jarque_bera_p}")
    print("\n----------------------------------------------\n")

# Plotting and visualization can be added here if needed
